### FILE UPLOAD TO SNOWFLAKE USING SQLALCHEMY

In [22]:
from sqlalchemy import create_engine, text

##### Functions to create snowflake connection, internal staging, creating table, staging data and loading staged data to Snowflake database

In [23]:
import configparser
config = configparser.ConfigParser()
config.read('../configuration.properties')

user = config['SNOWFLAKE']['User']
password = config['SNOWFLAKE']['Password']
account = config['SNOWFLAKE']['Account']
warehouse = config['SNOWFLAKE']['Warehouse']
database = config['SNOWFLAKE']['Database']
schema = config['SNOWFLAKE']['Schema']

# Create a connection string
connection_string = f'snowflake://{user}:{password}@{account}/' \
                        f'?warehouse={warehouse}&database={database}&schema={schema}'

In [24]:
def create_internal_stage(engine, stage_name):
    create_stage_query = f"""
    CREATE STAGE IF NOT EXISTS {stage_name};
    """
    with engine.connect() as connection:
        connection.execute(text(create_stage_query))

In [25]:
def create_table_with_csv_structure(engine, table_name):
    create_table_query = f"""
    CREATE OR REPLACE TABLE {table_name} (
        topic_name STRING,
        year STRING,
        level STRING,
        introduction STRING,
        learning_outcome STRING,
        summary STRING,
        summary_page_link STRING,
        pdf_file_Link STRING
    );
    """
    with engine.connect() as connection:
        connection.execute(text(create_table_query))

In [26]:
def put_data_into_stage(engine, csv_file_path, stage_name):
    put_data_query = f"""
    PUT file://{csv_file_path} @{stage_name};
    """
    with engine.connect() as connection:
        connection.execute(text(put_data_query))

In [27]:
def creating_file_format(engine, ff_name):
    create_ff_query = f"""
    CREATE OR REPLACE FILE FORMAT {ff_name}
    TYPE = 'CSV'
    FIELD_DELIMITER = '\t'
    SKIP_HEADER = 1
    SKIP_BLANK_LINES = True
    TRIM_SPACE = True;
    """
    with engine.connect() as connection:
        connection.execute(text(create_ff_query))

In [28]:
def load_data_from_stage_to_table(engine, table_name, stage_name, ff_name):
    copy_into_query = f"""
    COPY INTO {table_name} FROM @{stage_name} FILE_FORMAT = (FORMAT_NAME = {ff_name});
    """
    print(copy_into_query)
    with engine.connect() as connection:
        connection.execute(text(copy_into_query))

Steps to upload data to Snowflake Database:  
1. Create an engine for Snowflake connection  
2. Create an internal stage in Snowflake  
3. Create a table with reference to CSV structure  
4. Put data into stage  
5. Load data from stage to table using COPY INTO command

In [ ]:
# Create an engine for Snowflake Connection
engine = create_engine(connection_string)

# Create an internal stage in Snowflake
stage_name = 'refresher_readings_internal_stage'
create_internal_stage(engine, stage_name)

# Create a table
table_name = 'refresher_readings'
csv_file_path = '../web-scraping-and-dataset/scraped_data.csv'
create_table_with_csv_structure(engine, table_name)

# create file format
ff_name= 'refresher_data_ff'
creating_file_format(engine, ff_name)

# Stage the data
put_data_into_stage(engine, csv_file_path, stage_name)

# Load data from stage to table 
load_data_from_stage_to_table(engine, table_name, stage_name, ff_name)